# Our Data Sources

In [1]:
# Required Datasets
Sales_full = 'Sales_full'
Sales_mini = 'Sales_mini'
Current_price = 'Current_price'
COGs = 'COGs.xlsx'

# Our Libraries

In [2]:
# Standard libraries
import os
import io
import math
import calendar
import datetime
from os import listdir
from os.path import isfile, join
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

# Data manipulation & analysis
import pandas as pd
import numpy as np

# Excel file handling
import openpyxl
from openpyxl import load_workbook

# Google Colab environment
from google.colab import drive
from google.colab import auth

# Google API Client
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Google authentication
from google.auth import default
from google.oauth2 import service_account


# Mount to our driver

In [3]:
# First of all, we need to make sure that my directory is in My Drive
drive.mount('/content/drive')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content'

## Our Functions

In [4]:
def get_spreadsheet_id(url: str) -> str:
    """
    Extracts the Google Spreadsheet ID from a given URL.

    Args:
        url (str): Full Google Sheets URL.

    Returns:
        str: Spreadsheet ID.

    Raises:
        ValueError: If the URL is invalid or ID cannot be extracted.
    """
    try:
        return url.split("/d/")[1].split("/")[0]
    except (IndexError, AttributeError):
        raise ValueError("Invalid Google Sheets URL provided.")

In [5]:
def get_information_about_file_in_drive(file_id):
  # Get default credentials and build the Drive service
  auth.authenticate_user()
  creds, _ = default()
  drive_service = build('drive', 'v3', credentials=creds)
  # get the spreadsheet ID
  file_id = get_spreadsheet_id(url)
  # Try to access the file metadata
  try:
      file = drive_service.files().get(fileId=file_id, fields="id, name, mimeType, owners").execute()
      print(f"✅ You have access to the file: {file['name']}")
      print(f"📄 File ID: {file['id']}")
      print(f"👤 Owner: {file['owners'][0]['emailAddress']}")
      print(f"📁 Type: {file['mimeType']}")
  except HttpError as e:
      if e.resp.status == 404:
            print("❌ File not found.")
      elif e.resp.status == 403:
          print("❌ You do NOT have access to this file.")
      else:
        print(f"⚠️ Error occurred: {e}")

In [6]:
def get_sheet_names(file_path: str):
    """
    Get all sheet names from a local Excel file.
    """
    try:
        # Load the Excel file
        xlsx = pd.ExcelFile(file_path)

        # Get all sheet names
        sheet_names = xlsx.sheet_names

        # print the sheet names
        print(sheet_names)
    except Exception as e:
        print(f"Error reading file: {e}")
        return []


In [7]:
def Get_Previous_Months_With_Year(num_of_previous_months: int = 3) -> dict:
    """
    Generate a dictionary of previous months with their corresponding years.

    This function calculates the month names and years for a given number of
    months preceding the current date. The results are stored in a dictionary
    where:
        - Keys are full month names (e.g., "July").
        - Values are the corresponding years (e.g., 2025).

    Parameters
    ----------
    num_of_previous_months : int, optional
        Number of past months to include (default is 3).

    Returns
    -------
    dict
        Dictionary of month names (str) mapped to years (int).
        Example: {'July': 2025, 'June': 2025, 'May': 2025}

    Examples
    --------
    >>> Get_Previous_Months_With_Year(3)
    {'July': 2025, 'June': 2025, 'May': 2025}

    >>> Get_Previous_Months_With_Year(5)
    {'July': 2025, 'June': 2025, 'May': 2025, 'April': 2025, 'March': 2025}
    """
    now = datetime.now()
    months = {}

    for i in range(1, num_of_previous_months + 1):
        past_date = now - relativedelta(months=i)
        month_name = past_date.strftime("%B")
        year = past_date.year
        months[month_name] = year

    return months


In [8]:
def filter_df_by_previous_months(df, num_of_previous_months = 3):
    """
    Filters the input DataFrame to include only rows from the previous three months.

    Args:
        df_full (pd.DataFrame): The full dataset containing at least 'Month' and 'Year' columns.

    Returns:
        pd.DataFrame: Filtered DataFrame with rows from the previous three months.
    """
    dict_month_year = Get_Previous_Months_With_Year(num_of_previous_months)
    df_previous_months = pd.DataFrame()

    for month, year in dict_month_year.items():
        df_t = df[(df['Month'] == month) & (df['Year'] == year)]
        df_previous_months = pd.concat([df_previous_months, df_t], ignore_index=True)

    return df_previous_months


In [9]:
def get_unmapped_marketplaces(df, marketplace_col, mapping_dict, with_counts=False):
    """
    Find unmapped marketplaces in a DataFrame compared to a mapping dictionary.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame that contains marketplace data.
    marketplace_col : str
        Name of the column in df that stores marketplace values.
    mapping_dict : dict
        Dictionary mapping marketplace names to countries (or other values).
    with_counts : bool, optional
        If True, returns a dictionary of unmapped marketplaces with their counts.
        If False, returns only a set of unmapped marketplaces.

    Returns
    -------
    set or dict
        - If with_counts=False: set of unmapped marketplaces.
        - If with_counts=True: dict {marketplace: count}.
    """
    unique_marketplaces = set(df[marketplace_col].unique())
    mapped_marketplaces = set(mapping_dict.keys())

    # Difference → marketplaces in df but not in dict
    unmapped_marketplaces = unique_marketplaces - mapped_marketplaces

    if with_counts:
        counts = (
            df[df[marketplace_col].isin(unmapped_marketplaces)][marketplace_col]
            .value_counts()
            .to_dict()
        )
        return counts
    else:
        return unmapped_marketplaces


# Key Dictionaries for Important Data Mapping


In [10]:
dict_maketplace_country = { 'Amazon.co.uk': "UK",
                           'Amazon.de':'EU',
                            'Amazon.fr': 'EU',
                            'Amazon.es': 'EU',
                            'Amazon.it': 'EU',
                            'Amazon.nl': 'EU',
                            'Amazon.com.be': 'EU',
                            'Amazon.ie': 'EU',
                            'Amazon.pl':'EU',
                            'Amazon.com.tr': 'EU',
                            'Amazon.se': 'EU',
                            'Amazon.ae': 'EU',
                            'Amazon.com': 'US',
                            'Amazon.ca': 'CA',
                            'Amazon.com.mx': 'MX',
                            'Amazon.co.jp': 'JP'}





LTSF_Threshold= {'US':181, 'CA':181 ,'UK': 241,  'EU': 241, 'JP':271}


In [11]:
USA_columns = [(                       ' ',                      'Products'),
            (                     'USA',                           'SKU'),
            (                     'USA',                          'ASIN'),
            (           'ON HAND STOCK',        'Average Sales (7 days)'),
            (           'ON HAND STOCK',       'Average Sales (30 days)'),
            (           'ON HAND STOCK',               'Units in Amazon'),
            (           'ON HAND STOCK',       'Transit to AMZ from 3PL'),
            (           'ON HAND STOCK',                  'Units in AWD'),
            (           'ON HAND STOCK',               'Stocks with 3PL'),
            (           'ON HAND STOCK',                   'TOTAL UNITS')]


CA_columns =  [(      'Unnamed: 0_level_0',                   'Products'),
            (                      'CA',                        'SKU'),
            (                      'CA',                       'ASIN'),
            (           'Average Sales',                     '7 days'),
            (           'Average Sales',                    '30 days'),
            (           'ON HAND STOCK',            'Units in Amazon'),
            (           'ON HAND STOCK',   'Transit to AMZ from 3 PL'),
            (           'ON HAND STOCK',            'Stocks with 3PL'),
            (           'ON HAND STOCK',                'TOTAL UNITS')]


UK_columns = [(      'Unnamed: 0_level_0',                   'Products'),
            (                      'UK',                        'SKU'),
            (                      'UK',                       'ASIN'),
            (           'Average Sales',                     '7 days'),
            (           'Average Sales',                    '30 days'),
            (           'ON HAND STOCK',            'Units in Amazon'),
            (           'ON HAND STOCK',   'Transit to AMZ from 3 PL'),
            (           'ON HAND STOCK',            'Stocks with 3PL'),
            (           'ON HAND STOCK',                'TOTAL UNITS')]


EU_columns = [(      'Unnamed: 0_level_0',                    'PRODUCT'),
            (                      'EU',                        'SKU'),
            (                      'EU',                       'ASIN'),
            (           'Average Sales',                     '7 days'),
            (           'Average Sales',                    '30 days'),
            (           'ON HAND STOCK',            'Units in Amazon'),
            (           'ON HAND STOCK',   'Transit to AMZ from 3 PL'),
            (           'ON HAND STOCK',            'Stocks with 3PL'),
            (           'ON HAND STOCK',                'TOTAL UNITS')]



JP_columns = [(      'Unnamed: 0_level_0',                    'PRODUCT'),
            (                   'JAPAN',                        'SKU'),
            (                   'JAPAN',                       'ASIN'),
            (           'Average Sales',                     '7 days'),
            (           'Average Sales',                    '30 days'),
            (           'ON HAND STOCK',            'Units in Amazon'),
            (           'ON HAND STOCK',   'Transit to AMZ from 3 PL'),
            (           'ON HAND STOCK',            'Stocks with 3PL'),
            (           'ON HAND STOCK',                'TOTAL UNITS')]







columns_names_usa = ['Products', 'SKU', 'ASIN', 'Average Sales (7 days)', 'Average Sales (30 days)', 'Units in Amazon', 'Transit to AMZ from 3PL',  'Units in AWD',  'Stocks with 3PL', 'TOTAL UNITS']
columns_names_other_countries = ['Products', 'SKU', 'ASIN', 'Average Sales (7 days)', 'Average Sales (30 days)', 'Units in Amazon', 'Transit to AMZ from 3PL', 'Stocks with 3PL', 'TOTAL UNITS']

# Getting our Data


## Getting the active cells in the full stock file

>> We get this data from the full stock file and I have edit access to it

In [12]:
# -------------------------------
# Step 1. Get File Information
# -------------------------------

url = "https://docs.google.com/spreadsheets/d/1fGDIkxG_QDuQtZUKssmz399nWGntz56O/edit?usp=sharing&ouid=115826017476220798647&rtpof=true&sd=true"
get_information_about_file_in_drive(url)


# -------------------------------
# Step 2. Authenticate Google Drive API
# -------------------------------
auth.authenticate_user() # Google Colab authentication
creds, _ = default()  # Get default Colab credentials
drive_service = build('drive', 'v3', credentials=creds)


# -------------------------------
# Step 3. Extract File ID
# -------------------------------
file_id = get_spreadsheet_id(url)



# -------------------------------
# Step 4. Download Excel File
# -------------------------------
request = drive_service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)

done = False
while not done:
    status, done = downloader.next_chunk()
    print(f"🔄 Download progress: {int(status.progress() * 100)}%")

# Move to beginning of file for openpyxl
fh.seek(0)


# -------------------------------
# Step 5. Load Workbook
# -------------------------------
wb = openpyxl.load_workbook(fh, data_only=True)



# -------------------------------
# Step 6. Define Sheet Configurations
# Each country has:
# - list of columns to use
# - list of column names (for renaming)
# -------------------------------
df_fs = pd.DataFrame()
dict_of_countries = {
    'USA ': [USA_columns, columns_names_usa],
    'CA': [CA_columns, columns_names_other_countries],
    'UK ': [UK_columns, columns_names_other_countries],
    'EU ': [EU_columns, columns_names_other_countries],
    'JP': [JP_columns, columns_names_other_countries]
}


# -------------------------------
# Step 7. Iterate Sheets and Clean Data
# -------------------------------

for country, (columns_to_use, column_names) in dict_of_countries.items():
    ws = wb[country]

    # Read sheet rows (skip hidden rows, but keep header row)
    data = []
    for i, row in enumerate(ws.iter_rows(values_only=True), start=1):
        if i == 1 or not ws.row_dimensions[i].hidden:
            data.append(row)

    # Convert to DataFrame
    df_s = pd.DataFrame(data)

    # Remove non-standard rows and extra columns
    df_s  = df_s.iloc[3:, 0:len(pd.read_excel(fh, header=[0, 1 ], sheet_name= country).columns)] # remove unformal columns and rows
    df_s.columns = pd.read_excel(fh, header=[0, 1 ], sheet_name= country).columns # give the columns its original names
    df_s = df_s[columns_to_use]  # Keep only needed columns
    df_s.columns = column_names  # Rename columns
    df_s['Country'] = country    # Add country label
    df_s = df_s[df_s['ASIN'].isna() == False ] # keep only the filled columns

    # Append to final dataset
    df_fs = pd.concat([df_s, df_fs], ignore_index=True)




# -------------------------------
# Step 8. Final Cleanup
# -------------------------------

df_fs.fillna(0,inplace = True) # Replace NaN with 0
df_fs['Country'] = df_fs['Country'].str.strip().replace({'USA': 'US'})  # Normalize country codes
df_fs['ASIN'] = df_fs['ASIN'].str.strip()   # Clean ASIN values




# SKU and products name duplication cleaning
df_asin_sku = df_fs[['Products', 'ASIN', 'Country', 'SKU']].copy()

df_asin_sku_grouped = (
    df_asin_sku
    .groupby(["ASIN", "Country"])
    .agg({
        "SKU": lambda x: ", ".join(sorted(set(x.dropna().astype(str)))),
        "Products": lambda x: ", ".join(sorted(set(x.dropna().astype(str))))
    })
    .reset_index()
)



# exclude the out of stock products
df_fs = df_fs[df_fs['TOTAL UNITS'] > 0 ]

# Pivot for structured analysis (grouped by product, ASIN, and country)
df_fs = pd.pivot_table(df_fs, index = ['ASIN', 'Country'], values = ['Average Sales (7 days)',
       'Average Sales (30 days)', 'Units in Amazon', 'Transit to AMZ from 3PL',
       'Stocks with 3PL', 'TOTAL UNITS',  'Units in AWD'], aggfunc = 'sum').reset_index()

df_fs = df_fs.merge(df_asin_sku_grouped, how = 'left', on =["ASIN", "Country"])


✅ You have access to the file: Full stocks count.xlsx
📄 File ID: 1fGDIkxG_QDuQtZUKssmz399nWGntz56O
👤 Owner: annie@tilcotrading.com
📁 Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
🔄 Download progress: 100%


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/tmp/ipython-input-3976809241.py:97: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_fs.fillna(0,inplace = True) # Replace NaN with 0


### Cleaning the full stock data and make it ready for our report

In [13]:
# --- Step 1: Clean numeric fields ---
# Replace "#N/A" with NaN, then fill missing values with 0 and convert to float for safe numerical calculations
df_fs[['Average Sales (30 days)',
       'Average Sales (7 days)',
       'Stocks with 3PL',
       'TOTAL UNITS',
       'Transit to AMZ from 3PL',
       'Units in AWD',
       'Units in Amazon']] = (
    df_fs[['Average Sales (30 days)',
           'Average Sales (7 days)',
           'Stocks with 3PL',
           'TOTAL UNITS',
           'Transit to AMZ from 3PL',
           'Units in AWD',
           'Units in Amazon']]
    .replace("#N/A", np.nan)   # convert "#N/A" → NaN
    .fillna(0)                 # replace NaN with 0
    .astype(float)             # safely convert to float
)

# --- Step 2: Calculate stock duration ---
# Days stock lasts using 7-day and 30-day average sales
df_fs["Days stock last (7 days)"] = df_fs['TOTAL UNITS'] / df_fs['Average Sales (7 days)']
df_fs["Days stock last (30 days)"] = df_fs['TOTAL UNITS'] / df_fs['Average Sales (30 days)']

# --- Step 3: Handle errors ---
# Replace any remaining NaN with 0 and infinite values with 0
df_fs.fillna(0, inplace = True)
df_fs.replace({np.inf: 0, -np.inf: 0}, inplace = True)


# --- Step 4: Map thresholds by country ---
# Each country has its own LTSF threshold defined in LTSF_Threshold
df_fs['LTSF_Threshold'] = df_fs['Country'].map(LTSF_Threshold)

# --- Step 5: Flag products exceeding thresholds ---
# Compare stock duration against threshold for both 7-day and 30-day calculations
df_fs['7 days LTSF'] = df_fs['Days stock last (7 days)'] > df_fs['LTSF_Threshold']
df_fs['30 days LTSF'] = df_fs['Days stock last (30 days)'] > df_fs['LTSF_Threshold']

# Create final LTSF flag (True if either condition is met)
df_fs['LTSF'] = df_fs['7 days LTSF'] | df_fs['30 days LTSF']

# --- Step 6: Filter dataset ---
# Keep only rows where LTSF flag is True (i.e., long-term stock exists)
df_fs = df_fs[df_fs['LTSF'] == True]

/tmp/ipython-input-3158482333.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)                 # replace NaN with 0


>> Now we have the full stock dataframe **df_fs**, which contains the active sales data from the full stock file

## Getting the sales of our products

>> We obtain this data from the sales full file, which is an aggregated file generated by the collect-and-reprocess notebook.

In [14]:
# Define the root path where the final shaped sales files are stored
root_path = r'/content/drive/My Drive/Sales Data/Final Shape of Files'

# Change the working directory to the root path
os.chdir(root_path)

# Define the specific columns we need to import from the sales full file
df_sf_cols = ['Date','Marketplace','ASIN','SKU','SalesOrganic', 'SalesSponsoredProducts', 'SalesSponsoredDisplay', 'UnitsOrganic', 'UnitsPPC', 'FBALongTermStorageFee', 'FBAStorageFee', 'FBAPerUnitFulfillmentFee', 'NetProfit']

# Load the sales full file into a DataFrame with only the required columns
df_sf = pd.read_csv(Sales_full, usecols = df_sf_cols)

# Replace missing values (NaN) with 0 for consistency in calculations
df_sf.fillna(0, inplace = True)

# Convert 'Date' column to datetime format (day/month/year)
df_sf['Date'] = pd.to_datetime(df_sf['Date'], format = "%d/%m/%Y")

# Extract year and month from the 'Date' column for easier grouping/aggregation
df_sf['Year'] = df_sf['Date'].dt.year
df_sf['Month'] = df_sf['Date'].dt.month_name()

# Calculate new business metrics:
# Gross Revenue = organic + sponsored products + sponsored display sales
df_sf['Gross Revenue'] = df_sf['SalesOrganic'] +  df_sf['SalesSponsoredProducts'] + df_sf['SalesSponsoredDisplay']

# Units Sold = organic units + PPC units
df_sf['Units Sold'] = df_sf['UnitsOrganic'] + df_sf['UnitsPPC']

# FBA storage fee = long-term storage + regular storage + per-unit fulfillment
df_sf['FBA storage fee'] = df_sf['FBALongTermStorageFee'] + df_sf['FBAStorageFee'] +  df_sf['FBAPerUnitFulfillmentFee']

# Reorder and keep only the final relevant columns for reporting
df_sf = df_sf[['Date', 'Marketplace', 'ASIN','SKU',
       'Year','Month', 'Gross Revenue', 'Units Sold', 'FBA storage fee', 'NetProfit']]




In [15]:
# Filter the sales full dataframe to only include data from the previous month
df_sf_last_month = filter_df_by_previous_months(df_sf, num_of_previous_months = 1)

# Aggregate Gross Revenue and Net Profit by Marketplace, ASIN, Year, and Month
df_sf_last_month  = pd.pivot_table(df_sf_last_month, index = ['Marketplace', 'ASIN', 'Year', 'Month'], values = ['Gross Revenue', 'NetProfit'], aggfunc = 'sum').reset_index()

# Map each Marketplace to its corresponding Country using the predefined dictionary
df_sf_last_month['Country'] = df_sf_last_month['Marketplace'].map(dict_maketplace_country)

# Re-aggregate data by Country and ASIN (summing up revenue and profit)
df_sf_last_month = pd.pivot_table(df_sf_last_month, index = ['Country', 'ASIN'], values = ['Gross Revenue', 'NetProfit'], aggfunc = 'sum').reset_index()

# Calculate Net Margin = Net Profit ÷ Gross Revenue
df_sf_last_month['NetMargin'] = df_sf_last_month['NetProfit'] / df_sf_last_month['Gross Revenue']

# Replace missing values with 0 to ensure clean numerical results
df_sf_last_month.fillna(0, inplace = True)

# Replace infinite values (e.g., division by zero) with 0
df_sf_last_month.replace({np.inf: 0, -np.inf: 0}, inplace = True)



>> Now we have the full sales data of the previous month **df_sf_last_month** which contains the margins of our products in the last month

## Getting our products' prices

>> We obtain this data from the current price file, which is an aggregated file generated by the collect-and-reprocess notebook.

In [16]:
# Read the current price data file
df_cp = pd.read_csv(Current_price)

# Sort the data so the most recent price entries come first
df_cp.sort_values('Date', ascending = False, inplace = True)

# Keep only the latest price record for each unique ASIN–Region–Marketplace combination
price_list = df_cp.drop_duplicates(subset = ['ASIN','Region', 'Marketplace'], keep = 'first')

# Rename 'Region' column to 'Country' for consistency with other datasets
price_list.rename(columns = {'Region': 'Country'}, inplace = True)

# Keep only the relevant columns
price_list = price_list[['Marketplace', 'ASIN', 'Price']]

# Map each Marketplace to its corresponding Country
price_list['Country'] = price_list['Marketplace'].map(dict_maketplace_country)

# Get unmapped marketplaces with their counts
print(get_unmapped_marketplaces(price_list, 'Marketplace', dict_maketplace_country, with_counts=True))

# Aggregate prices by Country and ASIN (using mean in case multiple records exist)
price_list = pd.pivot_table(price_list, index = ['Country', 'ASIN'], values = ['Price'], aggfunc = 'mean').reset_index()

{}


/tmp/ipython-input-4119580494.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_list.rename(columns = {'Region': 'Country'}, inplace = True)


>> Now, we have the **price_list** dataframe, which contains the most recent price for each product in every country.

# Gathering our sources together


In [17]:
# ------------------------------------------------------------------
# Define the column order for the final dataframe to ensure a consistent structure
# ------------------------------------------------------------------
cols_order = ['Country',
    'Products', 'SKU', 'ASIN',
    'Average Sales (7 days)', 'Average Sales (30 days)',
    'Units in Amazon', 'Units in AWD',
    'Stocks with 3PL', 'Transit to AMZ from 3PL', 'TOTAL UNITS',
    'Days stock last (7 days)', 'Days stock last (30 days)',
    'Price', 'NetMargin', 'LTSF_Threshold',
    '7 days LTSF', '30 days LTSF', 'LTSF',
    'Gross Revenue', 'NetProfit'
]

# ------------------------------------------------------------------
# Step 1: Merge the full stock dataframe (df_fs) with the price list
# This adds the most recent products' pricing to the stock data
# ------------------------------------------------------------------
df_map = df_fs.merge(price_list, on=['Country', 'ASIN'], how='left')

# ------------------------------------------------------------------
# Step 2: Merge the result with last month's sales dataframe (df_sf_last_month)
# This produces df_full, which consolidates stock, pricing, and margins data
# ------------------------------------------------------------------
df_full = df_map.merge(df_sf_last_month, on=['Country', 'ASIN'], how='left').fillna(0)

# ------------------------------------------------------------------
# Step 3: Handle unavailable values in Price/NetMargin
# ------------------------------------------------------------------
df_full[["NetMargin", "Price"]] = (
    df_full[["NetMargin", "Price"]].replace({0: "Unavailable"})
)


# ------------------------------------------------------------------
# Step 4: Reorder dataframe columns
# ------------------------------------------------------------------
df_full = df_full[cols_order]

# ------------------------------------------------------------------
# Step 5: Display available countries
# ------------------------------------------------------------------
df_full['Country'].unique()


array(['CA', 'UK', 'US', 'EU', 'JP'], dtype=object)

In [18]:
# ------------------------------------------------------------------
# Save the final stock dataframe (df_full) into an Excel file.
# Each country will be written into a separate sheet for easier review.
# ------------------------------------------------------------------

# Set the root directory where the Excel file will be saved
root_path = r'/content/drive/My Drive/Sales Data/Stuck Stocked'
os.chdir(root_path)

# Write all sheets in one go
with pd.ExcelWriter("Stuck Stocked.xlsx", engine="openpyxl") as writer:
    # Save the full dataframe as a base sheet
    df_full.to_excel(writer, sheet_name="All Countries", index=False)

    # Loop through each unique country and save to its own sheet
    for country in df_full['Country'].unique():
        df_full[df_full['Country'] == country].to_excel(
            writer, sheet_name=country, index=False
        )
